In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

In [2]:
path = "https://www.leboncoin.fr/annonces/offres/aquitaine/?th=1&q=renault%20zo%E9"

# Now we get html code from page
soup = BeautifulSoup(urlopen(path), 'html.parser')
    
# We extract values and labels for the ones that are in bold in the html code 
extracted_href = soup.findAll("a", {"href": re.compile("//www.leboncoin.fr/voitures/*")})
extracted_href

[<a class="list_item clearfix trackable" data-info='{"event_type" : "selfpromotion", "campaign" : "ad_search", "ad_listid" : "1321806610", "ad_location" : "list_content", "ad_position" : "5", "ad_type" : "offres", "ad_offres" : "pro", "ad_options" : "|photosup|"}' href="//www.leboncoin.fr/voitures/1321806610.htm?ca=2_s" title="Renault Zoe Life">
 <!-- Listing item image -->
 <div class="item_image">
 <span class="item_imagePic">
 <span class="lazyload" data-imgalt="Renault Zoe Life" data-imgsrc="https://img5.leboncoin.fr/ad-thumb/3c0262f2275570facbae7353c8d41b3bd1ef1484.jpg" style="display:block; width:100%; height:100%;"></span>
 </span>
 <span class="item_imageNumber">
 <i class="icon-camera icon-2x nomargin"></i>
 <span>6</span>
 </span>
 </div>
 <!-- Save ad (Mes annonces) -->
 <div class="saveAd" data-savead-id="1321806610" title="">
 <div class="saveMsg"><i class="showTip mediumgrey icon-heart-outline icon-2x nomargin" data-position="left" data-text="Sauvegarder l'annonce"></i></

In [3]:
s = str(extracted_href)
links_and_titles = re.findall('//www.leboncoin.fr/voitures/*([^\n\r]*)', s)
a = list(map(lambda x: str.split(x, '" title="'), links_and_titles))
title = [item[1] for item in a]
links = re.findall(r'\/\/www.leboncoin.fr/voitures/\d+\.\w*.\w*.\w*', s)

In [4]:
path2 = "http://" + links[0][2:]
# Now we get html code from page
soup2 = BeautifulSoup(urlopen(path2), 'html.parser')
    
extracted_price = soup2.findAll("h2", {"class": re.compile("item_price clearfix")})
print(extracted_price)
price = re.findall(r'content="(.*?)"',str(extracted_price)) # price contient [prix, Currency]
extracted_year = soup2.findAll("span", {"itemprop": re.compile("releaseDate")})
year = extracted_year[0].text.strip()
# Année OK, modèle OK, kilométrage OK, prix OK, téléphone du propriétaire OK, est ce que la voiture est vendue par un professionnel ou un particulier.

[<h2 class="item_price clearfix" content="11490" itemprop="price">
<span class="property">Prix</span>
<meta content="EUR" itemprop="priceCurrency">
<span class="value">
                    
                            11 490 €
                        
                    
                    
                </span>
</meta></h2>]


In [5]:
extracted_description = soup2.findAll("div", {"class": "line properties_description"})
extracted_description = extracted_description[0]

In [6]:
km = re.findall(r'(\d+).?(\d*)\s*(km|kms)', str(extracted_description))
if(len(km)>0):
    elements_of_km = ""
    for item in km[0][:len(km[0])-1]:
        elements_of_km = elements_of_km + item
    km = elements_of_km
else:
    km="NA"
km

'NA'

In [7]:
model = re.findall(r'(zen|intens|life|Zen|Intens|Life)', str(extracted_description))

In [8]:

if(len(model)>0):
    model = model[0]
else:
    model = 'NA'

In [9]:
model

'Life'

In [10]:
phone_number = re.findall(r'(0|\+33|0033)[1-9][0-9]{8}', str(extracted_description))

In [11]:
if(len(phone_number)>0):
    phone_number = phone_number[0]
else:
    phone_number = "NA"
phone_number

'0'

In [12]:
ispro = soup.findAll("p", {"class": "item_supp"})

soup2 = BeautifulSoup(str(ispro), 'html.parser')
ispro2 = soup2.findAll("p", {"content": "Voitures"})


In [13]:
sellerispro = []
for item in ispro2:
    if "(pro)" not in str(item):
        sellerispro.append(False)
    else:
        sellerispro.append(True)
print(len(sellerispro))
print(len(links))
#print(len(titles))

25
25


In [14]:
# Function to find year, price, km, model, and phone number in one ad:
def find_informations_on_ad(path_to_ad):
    
    # Now we get html code from page
    soup2 = BeautifulSoup(urlopen(path_to_ad), 'html.parser')
    
    # We extract the price
    extracted_price = soup2.findAll("h2", {"class": re.compile("item_price clearfix")})
    price = re.findall(r'content="(.*?)"',str(extracted_price)) # price contient [prix, Currency]
    
    # We extract the year
    extracted_year = soup2.findAll("span", {"itemprop": re.compile("releaseDate")})
    year = extracted_year[0].text.strip()
    
    # Now we want to extract phone number, model, and kilometers of the car in the description:
    # We first extract the description and convert it to a string:
    extracted_description = soup2.findAll("div", {"class": "line properties_description"})
    extracted_description = str(extracted_description[0])
    
    # Extraction of the kilometers of the car:
    km = re.findall(r'(\d+).?(\d*)\s*(km|kms|KM|KMS|Km|Kms)', str(extracted_description))
    # This can find a lot of different values for the kilometers, we admit that the first one is the one we want.
    # But we check as well if we extracted at least one value for kilometers. If not we set it to NA
    if(len(km)>0):
        elements_of_km = ""
        for item in km[0][:len(km[0])-1]:
            elements_of_km = elements_of_km + item
        km = elements_of_km
    else:
        km="NA"
    
    # Extraction of the car's model, it can be between 3 values, Zen, Intens and Life.
    model = re.findall(r'(zen|intens|life|Zen|Intens|Life)', str(extracted_description))
    if(len(model)>0):
        model = model[0]
    else:
        model = 'NA'
        
    # Extraction of the seller's phone number:
    phone_number = re.findall(r'0[1-9]\d{8}', str(extracted_description))
    if(len(phone_number)>0):
        phone_number = phone_number[0]
    else:
        phone_number = "NA"
        
    return price, year, km, model, phone_number, str(extracted_description)

In [15]:
# Function to find all the cars in one page, and return a dataframe
def analyze_zoe(path):
    # Now we get html code from page
    soup = BeautifulSoup(urlopen(path), 'html.parser')
    
    # We extract the href corresponding to each car
    extracted_href = soup.findAll("a", {"href": re.compile("//www.leboncoin.fr/voitures/*")})
    
    s = str(extracted_href)
    
    # To have the titles corresponding to the links, we extract bot the links and titles for /voitures/
    # And store only the titles, because they are clean.
    titles = re.findall(r'title="(\S.*)"', s)
    titles = titles[:len(titles)-1]
    
    # title gathered one value more than it should have, it will always be the last one becaux it is in the page's footer.
    
    
    # Now we extract the links corresponding to cars ads. 
    links = re.findall(r'\/\/www.leboncoin.fr/voitures/\d+\.\w*.\w*.\w*', s)
    
    # We also check for each ad if this is a pro that it emmiting the ad or not.
    extract_item_supp = soup.findAll("p", {"class": "item_supp"})
    soup2 = BeautifulSoup(str(extract_item_supp), 'html.parser')
    ispro = soup2.findAll("p", {"content": "Voitures"})
    sellerispro = []
    for item in ispro:
        if "(pro)" not in str(item):
            sellerispro.append(False)
        else:
            sellerispro.append(True)
    prices = []
    years = []
    kms = []
    models = []
    phone_numbers = []
    descriptions = []
    for link in links:
        price, year, km, model, phone_number, description = find_informations_on_ad("http://" + link[2:])
        prices.append(price)
        years.append(year)
        kms.append(km)
        models.append(model)
        phone_numbers.append(phone_number)
        descriptions.append(description)
    print(titles)
    df = pd.DataFrame({"Car": titles, "Model": models,"Pro": sellerispro, "Price": prices, "Year":years, "km": kms, "Phone_number": phone_numbers, "Description": descriptions})
    return df

In [16]:
path = "https://www.leboncoin.fr/annonces/offres/aquitaine/?th=1&q=renault%20zo%E9"
df = analyze_zoe(path)
df

['Renault Zoe Life', 'Renault zoe life', 'ZOE Life 8990€', 'Renault Zoé Life pack Easy 2014', 'Renault Zoe LIFE CHARGE RAPIDE', 'Zoe zen', 'ZOE - 11 000 km-Excellent état', 'Renault Zoe ZEN', 'Renault Zoe LIFE CHARGE RAPIDE', 'Renault Zoe Intens Charge Rapide', 'Renault Zoe Zen', 'Renault Zoe LIFE', 'Renault Zoé life', 'Renault Zoe LIFE', 'Renault Zoe Zen', 'Renault zoe zen', 'Renault Zoe Life Charge Rapide Life Charge Rapide', 'Renault Zoe Zen charge rapide', 'Renault zoe', 'Renault Zoe LIFE CHARGE RAPIDE', 'Renault Zoe Life type 2', 'Renault Zoe LIFE', 'Renault Zoe ZEN', 'Renault Zoe LIFE', 'Renault Zoe LIFE']


,Car,Description,Model,Phone_number,Price,Pro,Year,km
0,Renault Zoe Life,"<div class=""line properties_description"">\n<p ...",Life,0558909010,"[11490, EUR]",True,2016,NA
1,Renault zoe life,"<div class=""line properties_description"">\n<p ...",NA,NA,"[9200, EUR]",False,2016,NA
2,ZOE Life 8990€,"<div class=""line properties_description"">\n<p ...",Life,NA,"[8990, EUR]",True,2014,38049
3,Renault Zoé Life pack Easy 2014,"<div class=""line properties_description"">\n<p ...",Life,NA,"[7800, EUR]",False,2014,40223
4,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",NA,0531081576,"[7490, EUR]",True,2014,NA
5,Zoe zen,"<div class=""line properties_description"">\n<p ...",NA,NA,"[9990, EUR]",True,2014,27545
6,ZOE - 11 000 km-Excellent état,"<div class=""line properties_description"">\n<p ...",Life,NA,"[9300, EUR]",False,2016,10500
7,Renault Zoe ZEN,"<div class=""line properties_description"">\n<p ...",NA,0531081574,"[9990, EUR]",True,2015,NA
8,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",NA,0531081574,"[9950, EUR]",True,2015,NA
9,Renault Zoe Intens Charge Rapide,"<div class=""line properties_description"">\n<p ...",Intens,0649173452,"[12000, EUR]",True,2015,NA


In [17]:
# Manque: Premier cleaning, remplir les NA modèles si présent dans Car


def check_title_model(title):
    model = re.findall(r'(zen|intens|life|Zen|Intens|Life|ZEN|INTENS|LIFE)', title)
    if(len(model)>0):
        return model[0]
    else:
        return 'NA'

    
    
df['Model'].loc[df['Model'] == 'NA'] = list(map(lambda x: check_title_model(x), df['Car'].loc[df['Model'] == 'NA']))

df

/Users/remi/Desktop/Cours/StarterKitBD/StarterKit_venv3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Car,Description,Model,Phone_number,Price,Pro,Year,km
0,Renault Zoe Life,"<div class=""line properties_description"">\n<p ...",Life,0558909010,"[11490, EUR]",True,2016,NA
1,Renault zoe life,"<div class=""line properties_description"">\n<p ...",life,NA,"[9200, EUR]",False,2016,NA
2,ZOE Life 8990€,"<div class=""line properties_description"">\n<p ...",Life,NA,"[8990, EUR]",True,2014,38049
3,Renault Zoé Life pack Easy 2014,"<div class=""line properties_description"">\n<p ...",Life,NA,"[7800, EUR]",False,2014,40223
4,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",LIFE,0531081576,"[7490, EUR]",True,2014,NA
5,Zoe zen,"<div class=""line properties_description"">\n<p ...",zen,NA,"[9990, EUR]",True,2014,27545
6,ZOE - 11 000 km-Excellent état,"<div class=""line properties_description"">\n<p ...",Life,NA,"[9300, EUR]",False,2016,10500
7,Renault Zoe ZEN,"<div class=""line properties_description"">\n<p ...",ZEN,0531081574,"[9990, EUR]",True,2015,NA
8,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",LIFE,0531081574,"[9950, EUR]",True,2015,NA
9,Renault Zoe Intens Charge Rapide,"<div class=""line properties_description"">\n<p ...",Intens,0649173452,"[12000, EUR]",True,2015,NA


In [18]:
# Prendre les derniers chiffres pour les kilomètres, en général c'est 5 chiffres...

def check_number_digits_km(km):
    if(km == 'NA'):
        return km
    else:
        s = str(km)
        if(len(s)> 5):
             s= s[-5:]
        return int(s)

df['km'] = list(map(lambda x: check_number_digits_km(x), df['km']))
df

,Car,Description,Model,Phone_number,Price,Pro,Year,km
0,Renault Zoe Life,"<div class=""line properties_description"">\n<p ...",Life,0558909010,"[11490, EUR]",True,2016,NA
1,Renault zoe life,"<div class=""line properties_description"">\n<p ...",life,NA,"[9200, EUR]",False,2016,NA
2,ZOE Life 8990€,"<div class=""line properties_description"">\n<p ...",Life,NA,"[8990, EUR]",True,2014,38049
3,Renault Zoé Life pack Easy 2014,"<div class=""line properties_description"">\n<p ...",Life,NA,"[7800, EUR]",False,2014,40223
4,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",LIFE,0531081576,"[7490, EUR]",True,2014,NA
5,Zoe zen,"<div class=""line properties_description"">\n<p ...",zen,NA,"[9990, EUR]",True,2014,27545
6,ZOE - 11 000 km-Excellent état,"<div class=""line properties_description"">\n<p ...",Life,NA,"[9300, EUR]",False,2016,10500
7,Renault Zoe ZEN,"<div class=""line properties_description"">\n<p ...",ZEN,0531081574,"[9990, EUR]",True,2015,NA
8,Renault Zoe LIFE CHARGE RAPIDE,"<div class=""line properties_description"">\n<p ...",LIFE,0531081574,"[9950, EUR]",True,2015,NA
9,Renault Zoe Intens Charge Rapide,"<div class=""line properties_description"">\n<p ...",Intens,0649173452,"[12000, EUR]",True,2015,NA


In [19]:
# Nettoyer un peu la description:
def clean_descprition(text):
    text = re.sub(r'<[^>]*>', " ", text)
    #text = text.replace("\n", " ")
    return text.upper()

df['Description'] = list(map(lambda x: clean_descprition(x), df['Description']))
df

,Car,Description,Model,Phone_number,Price,Pro,Year,km
0,Renault Zoe Life,"\n DESCRIPTION : \n RENAULT ZOE LIFE BERLINE,...",Life,0558909010,"[11490, EUR]",True,2016,NA
1,Renault zoe life,\n DESCRIPTION : \n MISE EN CIRCULATION 22/06...,life,NA,"[9200, EUR]",False,2016,NA
2,ZOE Life 8990€,\n DESCRIPTION : \n ZOE LIFE COULEUR GRIS 38...,Life,NA,"[8990, EUR]",True,2014,38049
3,Renault Zoé Life pack Easy 2014,\n DESCRIPTION : \n RENAULT ZOÉ LIFE - CARRO...,Life,NA,"[7800, EUR]",False,2014,40223
4,Renault Zoe LIFE CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...,LIFE,0531081576,"[7490, EUR]",True,2014,NA
5,Zoe zen,\n DESCRIPTION : \n ZOE ZEN IMMATRICULE: 17...,zen,NA,"[9990, EUR]",True,2014,27545
6,ZOE - 11 000 km-Excellent état,\n DESCRIPTION : \n PARTICULIER VENDS ZOE TYP...,Life,NA,"[9300, EUR]",False,2016,10500
7,Renault Zoe ZEN,"\n DESCRIPTION : \n RENAULT ZOE ZEN BERLINE, ...",ZEN,0531081574,"[9990, EUR]",True,2015,NA
8,Renault Zoe LIFE CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...,LIFE,0531081574,"[9950, EUR]",True,2015,NA
9,Renault Zoe Intens Charge Rapide,\n DESCRIPTION : \n RENAULT ZOE INTENS CHARGE...,Intens,0649173452,"[12000, EUR]",True,2015,NA


In [20]:
# Ajouter la colonne Argus


In [21]:
path_argus = "https://www.lacentrale.fr/cote-voitures-renault-zoe--2013-.html"
def extract_links_argus(path):
    soup3 = BeautifulSoup(urlopen(path_argus), 'html.parser')
    
    # We extract values and labels for the ones that are in bold in the html code 
    extracted_href = soup3.findAll("a", {"href": re.compile("cote-auto-renault-zoe-*")})
    links_argus = re.findall(r'cote-auto-renault-zoe-(.*).html', str(extracted_href))
    return links_argus
links_argus = extract_links_argus(path_argus)
links_argus

['intens+charge+rapide-2013',
 'intens+charge+rapide+type+2-2013',
 'life+charge+rapide-2013',
 'life+charge+rapide+type+2-2013',
 'zen+charge+rapide-2013',
 'zen+charge+rapide+type+2-2013']

In [22]:
path_cote = "https://www.lacentrale.fr/cote-auto-renault-zoe-" + links_argus[0] + ".html"

def get_cote(path):
    soup4 = BeautifulSoup(urlopen(path), 'html.parser')
    # We extract values and labels for the ones that are in bold in the html code 
    extracted_cote = soup4.findAll("span", {"class": "jsRefinedQuot"})
    extracted_model = soup4.findAll("span", {"class": "sizeC clear txtGrey7C sizeC"})

    return extracted_cote[0].text.strip(), extracted_model[0].text.strip()
get_cote(path_cote)

('8 504', 'INTENS CHARGE RAPIDE')

In [23]:
years = [2013, 2014, 2015, 2016, 2017]
model_years = []
cotes = []
models = []
for year in years:
    path_argus = "https://www.lacentrale.fr/cote-voitures-renault-zoe--" + str(year) + "-.html"
    links_argus = extract_links_argus(path_argus)
    
    for link in links_argus:
        model_years.append(year)
        cote, model = get_cote("https://www.lacentrale.fr/cote-auto-renault-zoe-" + link + ".html")
        cotes.append(cote)
        models.append(model)



In [24]:
df_argus = pd.DataFrame({"Year": model_years, "Model": models, "Cote": cotes})
df_argus

,Cote,Model,Year
0,8 504,INTENS CHARGE RAPIDE,2013
1,9 766,INTENS CHARGE RAPIDE TYPE 2,2013
2,9 256,LIFE CHARGE RAPIDE,2013
3,9 296,LIFE CHARGE RAPIDE TYPE 2,2013
4,9 282,ZEN CHARGE RAPIDE,2013
5,9 792,ZEN CHARGE RAPIDE TYPE 2,2013
6,11 473,INTENS CHARGE RAPIDE,2014
7,11 846,INTENS CHARGE RAPIDE TYPE 2,2014
8,11 604,LIFE CHARGE RAPIDE,2014
9,9 296,LIFE CHARGE RAPIDE TYPE 2,2014


In [25]:
# On créé pour argus une colonne modele en extrayant Zen, LIFE, on intens.
def get_first_word(sentence):
    split_sentence = sentence.split(" ")
    return split_sentence[0]
def delete_first_word(sentence):
    split_sentence = sentence.split(" ")
    if len(split_sentence) > 1:
        return ' '.join(split_sentence[1:])
    else:
        return sentence
df_argus["Gamme"] = list(map(lambda x: get_first_word(x), df_argus["Model"]))
df_argus["Model"] = list(map(lambda x: delete_first_word(x), df_argus['Model']))
df_argus

,Cote,Model,Year,Gamme
0,8 504,CHARGE RAPIDE,2013,INTENS
1,9 766,CHARGE RAPIDE TYPE 2,2013,INTENS
2,9 256,CHARGE RAPIDE,2013,LIFE
3,9 296,CHARGE RAPIDE TYPE 2,2013,LIFE
4,9 282,CHARGE RAPIDE,2013,ZEN
5,9 792,CHARGE RAPIDE TYPE 2,2013,ZEN
6,11 473,CHARGE RAPIDE,2014,INTENS
7,11 846,CHARGE RAPIDE TYPE 2,2014,INTENS
8,11 604,CHARGE RAPIDE,2014,LIFE
9,9 296,CHARGE RAPIDE TYPE 2,2014,LIFE


In [26]:
df['Model'] = list(map(lambda x: x.upper(), df['Model']))
df['Car'] = list(map(lambda x: x.upper(), df['Car']))
df

,Car,Description,Model,Phone_number,Price,Pro,Year,km
0,ZOE ZEN,\n DESCRIPTION : \n ZOE ZEN IMMATRICULE: 17...,ZEN,NA,"[9990, EUR]",True,2014,27545
1,ZOE - 11 000 KM-EXCELLENT ÉTAT,\n DESCRIPTION : \n PARTICULIER VENDS ZOE TYP...,LIFE,NA,"[9300, EUR]",False,2016,10500
2,ZOE LIFE 8990€,\n DESCRIPTION : \n ZOE LIFE COULEUR GRIS 38...,LIFE,NA,"[8990, EUR]",True,2014,38049
3,RENAULT ZOE LIFE CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...,LIFE,0531081574,"[9950, EUR]",True,2015,NA
4,RENAULT ZOE INTENS,\n DESCRIPTION : \n RENAULT ZOE INTENS BERLIN...,INTENS,0531081574,"[9990, EUR]",True,2016,NA
5,RENAULT ZOE INTENS CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE INTENS CHARGE...,INTENS,0649173452,"[12000, EUR]",True,2015,NA
6,RENAULT ZOE ZEN,"\n DESCRIPTION : \n RENAULT ZOE ZEN BERLINE, ...",ZEN,0564283208,"[10100, EUR]",True,2014,NA
7,RENAULT ZOE LIFE,"\n DESCRIPTION : \n RENAULT ZOE LIFE BERLINE,...",LIFE,0531081576,"[8990, EUR]",True,2014,NA
8,RENAULT ZOÉ LIFE,\n DESCRIPTION : \n ZOÉ LIFE BLEU AZUR M...,LIFE,NA,"[9500, EUR]",False,2014,48000
9,RENAULT ZOE ZEN,"\n DESCRIPTION : \n RENAULT ZOE ZEN BERLINE, ...",ZEN,0564283208,"[9500, EUR]",True,2013,NA


In [27]:
def check_cote(model, year, description):
    
    df_filtered = df_argus
    # On filtre sur l'année dans un premier temps
    df_filtered = df_filtered.loc[df_filtered['Year'] == int(year)].loc[df_filtered['Gamme'] == model]
    #print(df_filtered)
    # On filtre ensuite sur la gamme
    
    # Maintenant il faut vérifier si notre voiture est parmis l'un des modèles:
    count = 0
    for gamme in df_filtered['Model']:
        count = count +1
        #print(gamme)
        if gamme in description:
            #print(gamme)
            #print(count)
            return df_filtered['Cote'].loc[df_filtered["Model"] == gamme].values[0].strip()
    return 'NA'

df['Argus'] = list(map(check_cote, df["Model"], df['Year'], df['Description']))
df

,Car,Description,Model,Phone_number,Price,Pro,Year,km,Argus
0,ZOE ZEN,\n DESCRIPTION : \n ZOE ZEN IMMATRICULE: 17...,ZEN,NA,"[9990, EUR]",True,2014,27545,NA
1,ZOE - 11 000 KM-EXCELLENT ÉTAT,\n DESCRIPTION : \n PARTICULIER VENDS ZOE TYP...,LIFE,NA,"[9300, EUR]",False,2016,10500,13 190
2,ZOE LIFE 8990€,\n DESCRIPTION : \n ZOE LIFE COULEUR GRIS 38...,LIFE,NA,"[8990, EUR]",True,2014,38049,NA
3,RENAULT ZOE LIFE CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE LIFE CHARGE R...,LIFE,0531081574,"[9950, EUR]",True,2015,NA,11 265
4,RENAULT ZOE INTENS,\n DESCRIPTION : \n RENAULT ZOE INTENS BERLIN...,INTENS,0531081574,"[9990, EUR]",True,2016,NA,NA
5,RENAULT ZOE INTENS CHARGE RAPIDE,\n DESCRIPTION : \n RENAULT ZOE INTENS CHARGE...,INTENS,0649173452,"[12000, EUR]",True,2015,NA,11 629
6,RENAULT ZOE ZEN,"\n DESCRIPTION : \n RENAULT ZOE ZEN BERLINE, ...",ZEN,0564283208,"[10100, EUR]",True,2014,NA,NA
7,RENAULT ZOE LIFE,"\n DESCRIPTION : \n RENAULT ZOE LIFE BERLINE,...",LIFE,0531081576,"[8990, EUR]",True,2014,NA,NA
8,RENAULT ZOÉ LIFE,\n DESCRIPTION : \n ZOÉ LIFE BLEU AZUR M...,LIFE,NA,"[9500, EUR]",False,2014,48000,NA
9,RENAULT ZOE ZEN,"\n DESCRIPTION : \n RENAULT ZOE ZEN BERLINE, ...",ZEN,0564283208,"[9500, EUR]",True,2013,NA,NA


In [28]:
# Ajouter une colonne plus cher, moins cher que la moyenne.
# Parcourir les autres paths et les ajouter au dataframe.
list(df["Model"], df['Year'], df['Description'])

TypeError: list() takes at most 1 argument (3 given)

In [ ]:
df['Description'][1]